In [1]:
import pandas as pd
import math 



class CheckUtils():
    def __init__(self) -> None:
        pass
    
    # def checkValue(func):
    #     from functools import partial
    #     partial 
    @staticmethod
    def checkSeries(serdf, 
                 isNan = True, 
                 isEmpty = True, 
                 isInf = True,
                    isNotInstance = None,
                    isNotIn = False,
                    isNotInVal = None,
                    types = None
                 ):
        

        
        def _check_subsequent(x, 
                              isNotInstance = isNotInstance, 
                              isNan = isNan, 
                              isEmpty = isEmpty, 
                              isInf = isInf, 
                              isnotIn = isNotIn, 
                              isNotInVal = isNotInVal):
            out = False
            
            if isNotInstance is not None:
                x1 = (not isinstance(x, isNotInstance))
            else:
                x1 = False

            if isinstance(x, (int, float, complex)):
                x2 = pd.isna(x) if isNan else False
                x3 = math.isinf(x) if isInf else False
                x4 = False
            
            elif x == None:
                x2 = False
                x3 = False
                x4 = True if isEmpty else False
            
            else:
                x2 = False
                x3 = False
                x4 = (len(x) == 0) if isEmpty else False
                
            x5 = isNotInVal not in x if isNotIn else False

            return out | x1 | x2 | x3 | x4 | x5
        
        def postprocess(df, types):
            if types == "row":
                return df.map(lambda x: not x)
            elif types == "col":
                return df[df == False].index # columns
            
        if isinstance(serdf, pd.Series):
            return serdf.progress_map(lambda x: _check_subsequent(x))
        
        elif isinstance(serdf, pd.DataFrame):
            out  = serdf.map(lambda x: _check_subsequent(x)) # cellwise
            if types == "row":
                return  postprocess(out.apply(lambda x: any(x), axis = 1), "row")
            elif types == "column":
                return postprocess(out.apply(lambda x: any(x), axis = 0), "col")
            else:
                raise ValueError("check type is not specified. select among 'row' and 'column")
        
        else:
            raise ValueError("this is not either series or dataframe.")

    
    @staticmethod
    def isEmpty(dataframe) -> bool:
    
        if len(dataframe.index) == 0:
            return True
    
        return False

    

In [28]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [95]:
import pandas as pd
import numpy as np
import os
from torch import nn
from torch import optim
from torch.nn import MSELoss
import torch

class Processor():
    def __init__(self, base, url, types):
        
        raw_url = "/home/hyohyeongjang/dependency_bert/semantics_result/raw/"
        dir = os.listdir(raw_url)
        dir = [i for i in dir if i.split(".")[0].split("_")[-1] == types]

        model_types, attention_types = ['dependency', "normal"], ["front", 'back', "none"]

        out = [[],[]]
        for i in dir:
            for model_index, model_type in enumerate(model_types):

                for attention_index, attention_type in enumerate(attention_types):
                    if (attention_type == i.split("_")[2]) and (model_type in i.split("_")[1]):
                        out[model_index].append(raw_url+i)

        self.out = out

        out_df = []            
        for merges in out:
            df = pd.DataFrame()
            for idx, file in enumerate(merges):
                df_temp = pd.read_csv(file).drop(columns = "Unnamed: 0")
                if idx == 0:
                    df = df_temp
                else:
                    df = df.merge(df_temp, on = ['type', 'quant', 'frontness', "comple"])

            out_df.append(df)
            
        df = pd.concat(out_df, axis = 0).reset_index(drop = True)
        

        self.col_front = [f"f{i}" for i in range(1,10)] 
        self.col_back = [f"b{i}" for i in range(1,10)]
        self.col_full = [f"n{i}" for i in range(1,10)]
        self.col_whole = self.col_front + self.col_back + self.col_full
        
        self.df = df[['type', 'quant', 'frontness', "comple"] + self.col_whole]

        self.category = ['type', 'quant', 'frontness', "comple"]
        self.model_type = ['normal', 'dependency']
        self.quant_type = ["every", "less than 5", "more than 5", "no"]
        self.front_type = ["front", "back"]
        self.compl_type = ["and", "or"]
        
    
    def process(self, *, output_url, 
                want_col = "all", # analyze which columns 
                of_model = None, of_quant = None, of_front = None, of_compl = None,
                per_model = None, per_quant = None, per_front = None, per_compl = None): 
        # analyze per: 1) certain values in each type, 2) whole values in each type, 3) no criterias of the type
        # by should be subset of per
        
        if want_col == "all": want_col = self.col_front + self.col_back + self.col_full  
        else:                 want_col = want_col



        crit_of = []
        if of_model != None: crit_of.append("type")
        if of_quant != None: crit_of.append("quant")
        if of_front != None: crit_of.append("frontness")
        if of_compl != None: crit_of.append("comple")

        if of_model == True: of_model = self.model_type
        if of_quant == True: of_quant = self.quant_type
        if of_front == True: of_front = self.front_type
        if of_compl == True: of_compl = self.compl_type
        of_all = [i for i in [of_model, of_quant, of_front, of_compl] if i != None]

        filters = self.df.apply(lambda x: all(x[i] in j for i, j in zip(crit_of, of_all)), axis = 1)
        my_df = self.df.copy().loc[filters]

        # TODO: by_xx should be subset of per_xx

        crit_per = []
        if per_model == True: crit_per.append("type")
        if per_quant == True: crit_per.append("quant")
        if per_front == True: crit_per.append("frontness")
        if per_compl == True: crit_per.append("comple")
        crit_all = {"type": of_model, "quant": of_quant, "frontness": of_front, "comple": of_compl}
        crit_other = set(crit_all.keys()) - set(crit_per)
        crit_other = {key: crit_all[key] for key in crit_other}

        print(crit_other)
        
        out = my_df.groupby(crit_per).agg(lambda x: (self.operation_comple_compare(x, crit_other)))
        filters = CheckUtils.checkSeries(out, types = "column")
        
        

        return self.split_database(out[filters])
        

    def operation_comple_compare(self, x, crit_other):
        if x.name in crit_other.keys():
            return np.nan
        
        # return round(x.iloc[0] / x.iloc[1],2)
        if (x.iloc[0] == 0) and (x.iloc[1]  == 0):   return np.nan
        else:                                       return x.iloc[0] < 0.95 * x.iloc[1] # and < or

    def split_database(self, df):
        return (df[filter(lambda x: x.startswith("f"), df.columns)],
               df[filter(lambda x: x.startswith("b"), df.columns)],
               df[filter(lambda x: x.startswith("n"), df.columns)])

    def slice_db(self, *, output_url, 
                want_col = "all", # analyze which columns 
                of_model = None, of_quant = None, of_front = None, of_compl = None):
                
        # analyze per: 1) certain values in each type, 2) whole values in each type, 3) no criterias of the type
        # by should be subset of per
        
        if want_col == "all": want_col = self.col_front + self.col_back + self.col_full  
        else:                 want_col = want_col



        crit_of = []
        if of_model != None: crit_of.append("type")
        if of_quant != None: crit_of.append("quant")
        if of_front != None: crit_of.append("frontness")
        if of_compl != None: crit_of.append("comple")

        if of_model == True: of_model = self.model_type
        if of_quant == True: of_quant = self.quant_type
        if of_front == True: of_front = self.front_type
        if of_compl == True: of_compl = self.compl_type
        of_all = [i for i in [of_model, of_quant, of_front, of_compl] if i != None]

        filters = self.df.apply(lambda x: all(x[i] in j for i, j in zip(crit_of, of_all)), axis = 1)
        my_df = self.df.copy().loc[filters]

        return self.split_database_to_slice_db(my_df)               
    
    def split_database_to_slice_db(self, df):
        return (df[self.category+ list(filter(lambda x: x.startswith("f"), df.columns))],
               df[self.category+ list(filter(lambda x: x.startswith("b"), df.columns))],
               df[self.category+ list(filter(lambda x: x.startswith("n"), df.columns))])

    
class Dataloader():
    def __init__(self, dff, dfb, dfn):
        self.dff = dff[['f4','f5','f6']]
        self.dfb = dfb[['b4','b5','b6']]
        self.dfn = dfn[['n4','n5','n6']]
    
    def __len__(self):
        return len(self.dff)
    
    def __getitem__(self, idx):
        return self.dff.iloc[idx].values, self.dfb.iloc[idx].values, self.dfn.iloc[idx].values


class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.linear = nn.Linear(in_features=2, out_features=1, bias = False)

    def forward(self, inputs):
        return self.linear(inputs)
    
        
if __name__ == "__main__":
    base = "~/dependency_bert/semantics_result/processed/"
    url = "semantics_result.csv"

    wrapper = []
    for whole in ['FullofWhole', "FullofConcerns"]:
        processor = Processor(base, url, whole)
        # out = processor.slice_db(output_url="temp.csv", want_col="all", 
        #                   of_model=['normal'], of_quant=True, of_front=True, of_compl=True,
        #                   per_model=True, per_quant=True, per_front=True, per_compl=False)
        out_df = processor.slice_db(output_url="temp.csv", want_col="all", 
                                of_model=True, of_quant=True, of_front=True, of_compl=True)
        
        dataloader = Dataloader(out_df[0], out_df[1], out_df[2])

        model = ToyModel().to(device)
        optimizer = optim.Adam(model.parameters())
        criterion = MSELoss()


        result_weight = []
        for idx,i in enumerate(dataloader):
            f456, b456, n456 = i
            src = torch.tensor([f456,b456], dtype=torch.float32).transpose(1,0).to(device)
            tgt = torch.tensor(n456, dtype=torch.float32).view(-1,1).to(device)

            for _ in range(10000):
                
            
                optimizer.zero_grad()
                out = model(src)
                loss = criterion(out, tgt)
                loss.backward()
                optimizer.step()

            res_weight = model.linear.weight.cpu().detach().numpy().reshape(-1)
            res_weight = [round(i,2) for i in (res_weight / sum(res_weight)).tolist()]       
            result_weight.append(res_weight)
            print(f"{idx} done")
        
        x = pd.DataFrame(result_weight)
        x = pd.concat([out_df[0][processor.category], x], axis = 1)
        x = pd.concat([pd.Series([whole] * 32), x], axis = 1)
        wrapper.append(x)
    final = pd.concat(wrapper, axis= 0)
    

        



0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done


In [96]:
wrapper[0].columns = ["attention_window", "model_type", "quantifier_type", "frontedness", "frontedness", "connectives", "preceding_proportion", "reverse_proportaion"]
wrapper[0].iloc[:,[0,1,2,3,5,6,7]].to_csv("~/dependency_bert/semantics_result/processed/whole.csv")
wrapper[1].columns = ["attention_window", "model_type", "quantifier_type", "frontedness", "frontedness", "connectives", "preceding_proportion", "reverse_proportaion"]
wrapper[1].iloc[:,[0,1,2,3,5,6,7]].to_csv("~/dependency_bert/semantics_result/processed/partial.csv")
